In [40]:
import gymnasium as gym
import os
#import gym
import numpy as np

import collections
import pickle
import tqdm

from stable_baselines3.common.buffers import ReplayBuffer


import os
import random
import time
from dataclasses import dataclass

import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import tyro
from torch.distributions.categorical import Categorical
from torch.utils.tensorboard import SummaryWriter

from typing import Optional

import functools
import random
from copy import copy

import numpy as np
from gymnasium.spaces import Discrete, MultiDiscrete, Box, Dict

from pettingzoo import AECEnv



import gymnasium


from pettingzoo.utils import agent_selector, wrappers

from gymnasium.utils import EzPickle



from statistics import NormalDist

import pygame

from typing import Any , Generic, Iterable, Iterator, TypeVar
ActionType = TypeVar("ActionType")


In [41]:
from Utilities.new_models import *
import utils_gym
import env_model_class_2


from board_env import *



%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
from IPython.display import clear_output, display

In [50]:
import numpy as np
class Hero_agent(int):
    def init_properties(self,agent_count,phases):
        #self.draw_count = 0
        self.init_win_count_iter(agent_count)
        self.init_move_count_epi(phases)
        

        
    def init_win_count_iter(self,agent_count):
        self.count_dict = {i:0 for i in range(1,agent_count+1)}
        self.count_draw_dict = {i:0 for i in range(1,agent_count+1)}
        self.draw_territory_count = 0
    def init_move_count_epi(self,phases):
        self.bad_move_count = 0
        self.bad_move_phase_count = {i:0 for i in phases}
        self.move_count =  {i:0 for i in phases}        
    
    def model_def(self, model):
        self.model =model

    def action_predict(self,data):
        return self.model.action_predict(data)
    def save_models(self):
        self.model.save_models()
    
    #def model_forward_call(self,name,kwarg):
    #    return self.model_dict[name](**kwarg)
        

a = Hero_agent(1)
a.init_properties(3,[1,2,3])

In [70]:
a.bad_move_phase_count

{1: 0, 2: 0, 3: 0}

In [63]:
class DDQN_module:
    def __init__(self, qnet_config_dict, actor_config_dict,args,device,writer,run_name,agent):

        self.agent = agent
        self.run_name =run_name 
        self.actor_config_dict = actor_config_dict
        self.qnet_config_dict = qnet_config_dict
        self.args = args
        self.device = device
        self.writer = writer
        
        self.actor = Actor_ddqn(**self.actor_config_dict).to(self.device)
        self.qf1 = QNetwork(**self.qnet_config_dict).to(self.device)
        self.qf1_target = QNetwork(**self.qnet_config_dict).to(self.device)
        self.target_actor = Actor_ddqn(**self.actor_config_dict).to(self.device)
        
        
        self.target_actor.load_state_dict(self.actor.state_dict())
        self.qf1_target.load_state_dict(self.qf1.state_dict())
        self.q_optimizer = optim.Adam(list(self.qf1.parameters()), lr=self.args.learning_rate)
        self.actor_optimizer = optim.Adam(list(self.actor.parameters()), lr=self.args.learning_rate)

    def action_predict(self,data):
        return self.actor(data)

    def train_write(self,data,iteration,epoch):
        #data = rb.sample(self.args.batch_size)
        qf1_a_values, qf1_loss, actor_loss = self.train(data)
        self.write(qf1_a_values, qf1_loss, actor_loss,epoch,iteration)

    def train(self,data):


        
        with torch.no_grad():
            collected_t_next = data.next_observations[:,-1]
            next_state_actions = self.target_actor(data.next_observations[:,:-1])
            qf1_next_target = self.qf1_target(data.next_observations[:,:-1], next_state_actions)

            
            next_q_value = data.rewards.flatten() + (1 - data.dones.flatten()) * (self.args.gamma**(collected_t_next+1)).view(-1) * (qf1_next_target).view(-1)
    
        qf1_a_values = self.qf1(data.observations[:,:-1], data.actions).view(-1)
        qf1_loss = nn.functional.mse_loss(qf1_a_values, next_q_value)
        
        # optimize the model
        self.q_optimizer.zero_grad()
        qf1_loss.backward()
        self.q_optimizer.step()
        
        #if global_step % args.policy_frequency == 0:
        actor_loss = -self.qf1(data.observations[:,:-1], self.actor(data.observations[:,:-1])).mean()
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
    
        # update the target network
        for param, target_param in zip(self.actor.parameters(), self.target_actor.parameters()):
            target_param.data.copy_(self.args.tau * param.data + (1 - self.args.tau) * target_param.data)
        for param, target_param in zip(self.qf1.parameters(), self.qf1_target.parameters()):
            target_param.data.copy_(self.args.tau * param.data + (1 - self.args.tau) * target_param.data)
        return qf1_a_values, qf1_loss, actor_loss
        

    def write(self,qf1_a_values, qf1_loss, actor_loss,epoch,iteration):
        
        ind_epoch = epoch + (iteration-1)*self.args.update_epochs
        self.writer.add_scalar(f"losses/{self.agent}/qf1_values", qf1_a_values.mean().item(), ind_epoch)
        
        self.writer.add_scalar(f"losses/{self.agent}/qf1_loss", qf1_loss.item(), ind_epoch)
        self.writer.add_scalar(f"losses/{self.agent}/actor_loss", actor_loss.item(), ind_epoch)
        
    def save_models(self):
        newpath = r'./models/'+ self.run_name +'/'+str(self.agent)
        if not os.path.exists(newpath):
            os.makedirs(newpath)
        torch.save(self.actor.state_dict(), newpath+"/actor.pt")
        torch.save(self.qf1.state_dict(), newpath+"/qf1.pt")
        torch.save(self.qf1_target.state_dict(), newpath+"/qf1_target.pt")
        torch.save(self.target_actor.state_dict(), newpath+"/target_actor.pt")       


    def load_models(self):
        newpath = r'./models/'+ self.run_name +'/'+str(self.agent)
        self.actor.load_state_dict(torch.load(newpath+"/actor.pt"))
        self.qf1.load_state_dict(torch.load(newpath+"/qf1.pt"))
        self.qf1_target.load_state_dict(torch.load(newpath+"/qf1_target.pt"))
        self.target_actor.load_state_dict(torch.load(newpath+"/target_actor.pt")) 
        

                                          
                   

In [107]:








class Trainer:
    """
    ## Trainer
    """

    def __init__(self, Args,param_dict =dict({})
                 ):
        # #### Configurations

        self.args = Args()#tyro.cli(Args)
        self.param_dict = param_dict
        self.update_arg(param_dict=param_dict)

        
        #self.args.batch_size = int(self.args.num_envs * self.args.num_steps)
        self.args.minibatch_size = int(self.args.batch_size // self.args.num_minibatches)
        #self.args.num_iterations = self.args.total_timesteps // self.args.batch_size
        self.gam = self.args.gamma
        #self.args.minibatch_size = 256#128 
        self.num_steps = self.args.num_steps#120000#1000000
        self.num_iterations = self.args.num_iterations
        self.episode_time_lim = self.args.episode_time_lim
        self.hero_agent_count = self.args.hero_agent_count
        self.env_config = self.args.env_config

        
        #self.env_config = dict(render_mode = 'rgb_array', default_attack_all  = True,
        #                    agent_count  = 4
        #                       ,use_placement_perc=True,render_=False)        
        
        self.run_name = f"{self.args.env_id}__{self.args.exp_name}__{self.args.seed}__{int(time.time())}"

        

        TB_log = True
        if TB_log:    
            self.writer = SummaryWriter(f"runs/{self.run_name}")
            self.writer.add_text(
                "hyperparameters",
                "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|" for key, value in vars(self.args).items()])),
            )
        
        # TRY NOT TO MODIFY: seeding
        random.seed(self.args.seed)
        np.random.seed(self.args.seed)
        torch.manual_seed(self.args.seed)
        
        torch.backends.cudnn.deterministic = self.args.torch_deterministic
        
        self.device = torch.device("cuda" if torch.cuda.is_available() and self.args.cuda else "cpu")
        
        
        self.playe_r = 1#"agent_1" #
        

        
        self.action_shape = (2,)



        self.env = env_risk(**self.env_config)
        
        self.env.reset(seed=42)


            

        
        self.total_agents  = len(self.env.possible_agents)
        self.total_phases = len(self.env.phases)
        
        sample_obs = self.obs_converter(torch.tensor(self.env.last()[0]['observation']),
                                        num_classes = self.total_agents+1
                                       )
        
        self.ob_space_shape = sample_obs.shape #env.observation_space(playe_r)['observation'].shape
        self.action_mask_shape = self.env.observation_space(self.playe_r)['action_mask'].shape
        
        #self.device = torch.device("cuda" if torch.cuda.is_available() and args.cuda else "cpu")

        
        
        self.agent_list = list(self.env.possible_agents)
        
        
        
        self.num_episodes = 10
       
        self.the_hero_agent = 1

        
        self.qnet_config_dict = dict(action_space = self.env.action_space(self.playe_r
                                                                         ).shape[0],
                                    ob_space=np.prod(self.ob_space_shape
                                                    )+np.prod(self.action_mask_shape)
                                         +1*( self.total_agents -1) #the current_agent +1#who actor agent was
                                         +1*(self.total_phases -1)#the current phase
                                         +1 # the number of troops
                               )
        self.actor_config_dict =  dict(env=self.env,
                        action_space = self.env.observation_space(self.playe_r)['action_mask'].shape[0],
                        ob_space=np.prod(self.ob_space_shape)+np.prod(self.action_mask_shape)
                                         +1*( self.total_agents-1) #the current_agent +1#who actor agent was
                                         +1*(self.total_phases -1)#the current phase
                                         +1 # the number of troops
                               )
        
        
        self.hero_agents_list = {i:Hero_agent(i) for i in range(1,self.hero_agent_count+1) } # this is a list , need to pass it as an argument
        
        for i in self.hero_agents_list:
            self.hero_agents_list[i].init_properties(self.total_agents,self.env.phases)        
        
            self.hero_agents_list[i].model_def(model = DDQN_module( self.qnet_config_dict, self.actor_config_dict,
                                                                   self.args,device = self.device,writer=self.writer,run_name=self.run_name,agent=i)
                                                )
            

            #self.target_actor.load_state_dict(self.actor.state_dict())
            #self.qf1_target.load_state_dict(self.qf1.state_dict())
            #self.q_optimizer = optim.Adam(list(self.qf1.parameters()), lr=self.args.learning_rate)
            #self.actor_optimizer = optim.Adam(list(self.actor.parameters()), lr=self.args.learning_rate)

    def update_arg(self,param_dict=dict({})):
       for i,j in param_dict.items():
           setattr(self.args,i,j)

        

    def obs_converter(self,  data, num_classes = 4, col =0 ):

        if col != None:
            return torch.concat((nn.functional.one_hot(data[:,col].detach().long(), 
                                                        num_classes = num_classes),
                                      data[:,~col,None]
                                ),axis=1
                               )[:,1:].to(self.device)
    
    def map_agent_phase_hot(self, data,num_classes = 3):
        return nn.functional.one_hot(torch.tensor(data),num_classes = num_classes)[1:].to(self.device)
    
    def map_agent_phase_vector(self, data,num_classes = 3):
        return nn.functional.one_hot(data[:,0].long(), 
                                                            num_classes = num_classes)[:,1:].to(self.device)


    def train_loop_init(self):
        self.gamma_t = {i:0 for i in self.env.possible_agents}
        
        
        self.draw_count = 0

        for i in self.hero_agents_list:
            self.hero_agents_list[i].init_win_count_iter(self.total_agents )
        
        
        #self.first_count = 0
        #self.second_count = 0
        #self.third_count = 0
        #self.third_count_draw = 0
        
        self.start_time = time.time()
        self.global_step = 0
        #self.faulting_player = ""

    
    
    def run_training_loop(self):
        """
        ### Run training loop
        """

        # last 100 episode information
        #tracker.set_queue('reward', 100, True)
        #tracker.set_queue('length', 100, True)

        obs = torch.zeros((self.num_steps,) + self.ob_space_shape).to(self.device)
        actions = torch.zeros((self.num_steps, ) + self.action_shape).to(self.device)
        action_masks = torch.zeros((self.num_steps, ) + self.action_mask_shape).to(self.device)
        current_agent = torch.ones((self.num_steps,1)).to(self.device)*0#-1
        current_phase = torch.zeros((self.num_steps,1)).to(self.device)
        current_troops_count = torch.zeros((self.num_steps,self.total_agents)).to(self.device)
        logprobs = torch.zeros((self.num_steps, )).to(self.device)
        rewards = torch.zeros((self.num_steps, self.total_agents)).to(self.device)
        rewards_2 = torch.zeros((self.num_steps, self.total_agents)).to(self.device)
        dones = torch.zeros((self.num_steps, self.total_agents)).to(self.device)
        values = torch.zeros((self.num_steps,  )).to(self.device)
        episodes = torch.ones((self.num_steps, )).to(self.device)*-1
        t_next = torch.zeros((self.num_steps, self.total_agents)).to(self.device)


        for i in self.hero_agents_list: # each agent has his own buffer, this is kinda pain because now this information is stored and not discarded
    
            self.hero_agents_list[i].rb = ReplayBuffer(
                    self.args.buffer_size,
                    Box(low =0, high=2000, shape =(self.qnet_config_dict['ob_space']+1,), dtype=np.float32),
                    Box(low =0, high=2000, shape =(2,), dtype=np.float32),
                    self.device,
                    handle_timeout_termination=False,
                )

        env = env_risk(**(self.env_config | {"render_mode" : None, "bad_mov_penalization" : 0.01,"render_":False}))
        env.reset(42)
        
        self.train_loop_init()
        
        for iteration in range(1, self.num_iterations+1):

            obs = torch.zeros((self.num_steps,) + self.ob_space_shape).to(self.device)
            actions = torch.zeros((self.num_steps, ) + self.action_shape).to(self.device)
            action_masks = torch.zeros((self.num_steps, ) + self.action_mask_shape).to(self.device)
            current_agent = torch.ones((self.num_steps,1)).to(self.device)*0#-1
            current_phase = torch.zeros((self.num_steps,1)).to(self.device)
            current_troops_count = torch.zeros((self.num_steps,self.total_agents)).to(self.device)
            logprobs = torch.zeros((self.num_steps, )).to(self.device)
            rewards = torch.zeros((self.num_steps, self.total_agents)).to(self.device)
            rewards_2 = torch.zeros((self.num_steps, self.total_agents)).to(self.device)
            dones = torch.zeros((self.num_steps, self.total_agents)).to(self.device)
            #values = torch.zeros((self.num_steps,  )).to(self.device)
            episodes = torch.ones((self.num_steps, )).to(self.device)*-1
            t_next = torch.zeros((self.num_steps, self.total_agents)).to(self.device)








            
            self.sample(
                                env,iteration,
                                obs
                                ,actions
                                ,action_masks
                                ,current_agent
                                ,current_phase
                                ,current_troops_count
                                ,logprobs
                                ,rewards
                                ,rewards_2
                                ,dones
                                ,values
                                ,episodes
                                ,t_next
                                
                 
                            )
            
            self.train(iteration)


            
            
            if self.global_step%100 ==0:
                SPS = int(self.global_step / (time.time() - self.start_time))
                print("SPS:", SPS)       
                self.writer.add_scalar("charts/SPS", SPS, self.global_step)
        
            
            self.save_models()


    
    def train(self,iteration):
        
        for epoch in range(self.args.update_epochs):
            
            if self.global_step > self.args.learning_starts:

                for i in self.hero_agents_list:
                    self.hero_agents_list[i].model.train_write(self.hero_agents_list[i].rb.sample(self.args.batch_size)
                                                         ,iteration,epoch)
                    
    def save_models(self):
        for i in self.hero_agents_list:
            self.hero_agents_list[i].save_models()  

    def reset_moves_hero_agents(self):
        for i in self.hero_agents_list:
            self.hero_agents_list[i].init_move_count_epi(self.env.phases)
    
    def sample(self,env,iteration,obs
                                ,actions
                                ,action_masks
                                ,current_agent
                                ,current_phase
                                ,current_troops_count
                                ,logprobs
                                ,rewards
                                ,rewards_2
                                ,dones
                                ,values
                                ,episodes
                                ,t_next
                                ):
        

        
        
        with torch.no_grad():
            # sample `worker_steps` from each worker
            #there are no worker steps... rather there are full episodes

            step = 0
            fault_condition = False
            clear_output(wait=True)
            phase = 0
        
            
            for episode in range(self.num_episodes):#num_episodes):
                
                total_rewards = {i:0 for i in env.possible_agents} #i can report this
                action=1
                
                if fault_condition:
                    env = env_risk(**(self.env_config | {"render_mode" : None,"bad_mov_penalization" : 0.01,"render_":False#False
                                                        })
                                      )#game.env(render_mode=None)

                curren_epi = episode + (iteration-1)*self.num_episodes
                env.reset(curren_epi) #for riplication
                
                fault_condition = False
                step_count = 0
                
                self.reset_moves_hero_agents()
                is_draw = 0
                
                #draw_territory_count = 0
                #is_third = 0

                for agent in env.agent_iter():
                    e_t = env.terminations
                    if sum(e_t.values()) <(self.total_agents-1):
                        observation, reward, termination, truncation, info = env.last()
        
                        observation['observation'] =  self.obs_converter(
                                                        torch.tensor(
                                                            observation['observation']
                                                        ).to(self.device,dtype=torch.float32),
                                                        num_classes = self.total_agents+1)
                        
                        episodes[step] = curren_epi
                        obs[step] = observation['observation']#torch.Tensor(observation['observation']).to(self.device) #sould i not add it .... meaning this is the last observation after the player dies
                        action_masks[step] = torch.Tensor(observation['action_mask']).to(self.device)
                        
                        #curr_agent = agent#int(agent[-1])
                        current_agent[step] = curr_agent = agent
                        current_phase[step] = phase = env.phase_selection
                        phase_mapping = self.map_agent_phase_hot(phase,num_classes = self.total_phases).float()
                        
                        curr_agent_mapping = self.map_agent_phase_hot(int(curr_agent)-1,
                                                                      num_classes = self.total_agents 
                                                                     ).float()
                        
                        current_troops_count[step] = torch.Tensor([env.board.agents[i].bucket for i in env.possible_agents]).to(self.device)
                    

                        model_in = torch.Tensor(torch.hstack((observation['observation'].reshape(-1),torch.tensor(observation['action_mask'].reshape(-1)).to(self.device),
                                           phase_mapping,
                                            curr_agent_mapping,
                                           torch.tensor([env.board.agents[curr_agent].bucket ]).to(self.device)))[None,:]#.repeat(3,axis = 0)
                                                ).float()
                        
                        #if e_t[curr_agent]:
                            #print('heeee')
                            
                        if termination or truncation: #this never happens ... the agent is removed from the current agent list and processed after the end of the cycle
                            
                            action = None

                            act = self.actor(torch.Tensor(model_in).to(self.device))
                            #act = self.
                            #act, logprob, _, value = agent_mod.get_action_and_value(model_in)
                            #values[step] = value.flatten() # so even if we are removing the guy ... we need to know what is the action he would 
                                                                #have taken at this point and what would have been its value
                            actions[step] = act #even after going what would have been
                            #logprobs[step] = logprob        
                        else:
                            mask = observation["action_mask"]
                            if (self.global_step < self.args.learning_starts) or (
                                np.random.rand() > min(
                                                ((curren_epi)/((self.num_iterations*self.num_episodes)/10))
                                                , 0.95)
                                                #) or (agent != self.the_hero_agent) 
                                                )or ( agent not in self.hero_agents_list):
        
                                
                                action = env.action_space(agent).sample()
                                #part_0 =np.random.choice(np.where(env.board.calculated_action_mask(agent))[0])
                                part_0 =np.random.choice(np.where(observation['action_mask'])[0])
                                action = torch.Tensor([[[part_0],[np.around(action[1],2)]]]).to(self.device)
                                action = action[:,:,0]
                            else:
                                action = self.hero_agents_list[curr_agent].action_predict(torch.Tensor(model_in).to(self.device))
                                #action = self.actor(torch.Tensor(model_in).to(self.device))
                            actions[step] = action
                            curr_agent_ = int(curr_agent)
        
                            if not observation['action_mask'][action[:,0].long()]: 
                                fault_condition =True
                                #self.faulting_player = agent

                                


                                if  curr_agent_ in self.hero_agents_list:
                                    self.hero_agents_list[curr_agent_].bad_move_count+=1
                                    self.hero_agents_list[curr_agent_].bad_move_phase_count[int(current_phase[step][0])]+=1  # when is the where_is_it_performing_bad_really
                                    #print('here',agent, action, observation['action_mask'])
                            
        
                            if  curr_agent_ in self.hero_agents_list:
                                self.hero_agents_list[curr_agent_].move_count[int(current_phase[step][0])]+=1  
                            #if self.the_hero_agent == curr_agent:
                                #move_count[int(current_phase[step][0])]+=1        
        
        
                        #print('here',agent, action)
                        if action != None :
                            act_2 = action.detach().cpu().numpy()[0]#list([action.detach().cpu().numpy()[0][0], max(action.detach().cpu().numpy()[0][1],0.1) ])
                            act_2 = list([act_2[0], max(act_2[1],0.001) ])
                        else:
                            act_2 = action
                            
                        env.step(act_2 if action != None else None)        
        
        
                        if action == None:
                            print('heeee')
                            rewards[step] = np.zeros(self.total_agents) # should i keep it -1? .... hm i dont think so .
                            dones[step] = np.zeros(self.total_agents) # frankly the guys is already done so we really dont have to do anything here.... this is the state post termination for a loser 
                            # but btw this is for the next agent ... action == None means in the last action the previous agent would have been removed.
                            #values[step] = 
                        else:
        
                            rewards_2[step] = torch.Tensor([env.curr_rewards[i] for i in env.possible_agents]).to(self.device)
                            curr_reward_list =  env.curr_rewards
                            
                            if (step_count == (self.episode_time_lim-1)):
                                is_draw=1
                                curr_reward_list = {i:-100 for i in env.possible_agents }
        
                            for i in env.possible_agents:
                                if i != curr_agent:
                                    self.gamma_t[i]+=1
                                else:
                                    self.gamma_t[i] =0
        
                                if (step_count == (self.episode_time_lim-1)):
                                    cr_rew = -100
                                    term = True
                                else:
                                    cr_rew = env.curr_rewards[i]
                                    term = env.terminations[i]

                                next_step_ = step-self.gamma_t[i]
                                rewards[next_step_,i-1] += (self.args.gamma**self.gamma_t[i])*cr_rew
                                t_next[next_step_,i-1] = self.gamma_t[i]
                                dones[next_step_,i-1] = torch.Tensor([term]).to(self.device) #so the panetly has to be added but attributions is really difficult
        
                        #list_curr_reward_list = np.array(list(curr_reward_list.values()))
                        
                        #if sum(curr_reward_list.values()) == -300:
                            #print('here')
                            #is_draw=1
        
                        
                        for age_i in env.possible_agents:
                            
                            total_rewards[age_i]+=curr_reward_list[age_i] #env.curr_rewards[age_i] if (step_count != episode_time_lim) else -100
                                    
                        
                        step +=1
                        self.global_step+=1
        
                    else:
                        print('done:',env.terminations,#env.terminations.values(),
                              ",total_reward:",total_rewards, ',iteration:',iteration,",episode:", episode )
                        break    
                
        
        
        
                    step_count+=1
                    
                    if (self.global_step == self.num_steps) :# or (fault_condition and (fa ulting_player != agent) and (len(env.agents)==0)):
                        break
                    elif (step_count == self.episode_time_lim):
                        break
                        
                #print(rewards[step-2])
                if self.global_step == self.num_steps:
                    break 

                for i in self.hero_agents_list:
                    self.hero_agents_list[i].position =self.total_agents
                    
                #[ position = 3 for i in ] 
                for k_,(i_,j_) in enumerate(sorted([(j_,i_) for i_,j_ in total_rewards.items()],reverse=True) 
                      ):
                    if int(j_) in self.hero_agents_list:
                        self.hero_agents_list[int(j_)].position = k_+1
                        
                        
                    #if j_==self.the_hero_agent:
                    #    position = k_+1

                cur_epi_list = (episodes == curren_epi)
                
                self.write_exploring(is_draw,#position,
                        curren_epi,step,
                        total_rewards,#bad_move_count
                        #,bad_move_phase_count,
                        #move_count,
                        observation,
                        env,
                        cur_epi_list)



            

            t_range = torch.Tensor(np.arange(0,step)).to(self.device,dtype=torch.int)
            #hero_steps = [current_agent == self.the_hero_agent][0][:,0][:step]
            
            #next_indecies = (t_next[:step,self.the_hero_agent-1].to(dtype=torch.int) + t_range +1)[hero_steps].long()
        
            #selected_t_next = t_next[:,self.the_hero_agent-1,None]#t_next[:step,0,None][hero_steps]
            infos = [dir({})]*step #t_next[:step,0]
            b_obs_a_all = torch.concat(( obs.reshape(-1,np.prod(self.ob_space_shape)) ,
                                    action_masks.reshape(-1,np.prod(self.action_mask_shape)),
                                    self.map_agent_phase_vector(current_agent,num_classes=self.total_agents+1)[:,1:],
                                    
                                    #map_agent_phase_vector(current_agent,num_classes=len(env.possible_agents)),
                                    self.map_agent_phase_vector(current_phase,num_classes=self.total_phases)#,
                                    #current_troops_count[:,self.the_hero_agent-1,None],
                                    #selected_t_next
                                   ),axis =1)

            self.update_hero_rb(b_obs_a_all,t_next,t_range,current_agent,actions,
                                rewards,dones,infos,current_troops_count,
                                step)

            
            #for i in zip(b_obs_a[:step][hero_steps].cpu().to(dtype=torch.float), b_obs_a[next_indecies].cpu().to(dtype=torch.float), 
            #   actions[:step][hero_steps].cpu().to(dtype=torch.float32), rewards[:step][hero_steps][:,self.the_hero_agent-1,None].cpu(), 
            #   dones[:step][hero_steps][:,self.the_hero_agent-1,None].cpu(), infos):
            #    rb.add(*i)


        avg_episode_length = torch.mean(torch.tensor(
                            [(episodes[:step] == i_epi).sum() for i_epi in episodes[:step].unique()]).float())#np.mean([(episodes[:step] == i_epi).sum() for i_epi in episodes[:step].unique()])
        
        self.writer.add_scalar("charts/avg_episodic_length", avg_episode_length, self.global_step)



        
        #return rb

    def update_hero_rb(self,b_obs_a_all,t_next,t_range,current_agent,actions,
                                rewards,dones,infos,current_troops_count,
                                step):



            for agent in self.hero_agents_list:
                #the_hero_agent = agent
                
                
                hero_steps = [current_agent == agent][0][:,0][:step]
                next_indecies = (t_next[:step,agent-1].to(dtype=torch.int) + t_range +1)[hero_steps].long()
                selected_t_next = t_next[:,agent-1,None]#t_next[:step,0,None][hero_steps]

                b_obs_a = torch.concat((b_obs_a_all,
                                        current_troops_count[:,agent-1,None],
                                        selected_t_next),axis =1)
                
                for i in zip(b_obs_a[:step][hero_steps].cpu().to(dtype=torch.float), b_obs_a[next_indecies].cpu().to(dtype=torch.float), 
                   actions[:step][hero_steps].cpu().to(dtype=torch.float32), rewards[:step][hero_steps][:,agent-1,None].cpu(), 
                   dones[:step][hero_steps][:,agent-1,None].cpu(), infos):
                    self.hero_agents_list[agent].rb.add(*i)
        



        
        

    def write_exploring(self,is_draw,#position,
                        curren_epi,step,
                        total_rewards,#bad_move_count
                        #,bad_move_phase_count,
                        #move_count,
                        observation,
                        env,
                        cur_epi_list):

        if is_draw:
            self.draw_count +=1

            for i in self.hero_agents_list:

                self.writer.add_scalar(f"draw_charts_agent_{i}/position_draw",self.hero_agents_list[i].position
                                                                                            ,self.draw_count)




                
                self.hero_agents_list[i].draw_territory_count = int(observation['observation'][:,i].sum())
                
                self.hero_agents_list[i].count_draw_dict[
                                                        self.hero_agents_list[i].position
                                                        ] +=1
                self.writer.add_scalar(f"draw_charts_agent_{i}/draw_territory_count",
                                                                               self.hero_agents_list[i].draw_territory_count,
                                                                               self.draw_count)#self.global_step)
                
                for j in self.hero_agents_list[i].count_draw_dict:
                    self.writer.add_scalar(f"draw_charts_agent_{i}/{j}_position_prop_draw",int(
                                                                                            self.hero_agents_list[i].position==j
                                                                                            ),self.draw_count)

                    if j not in [1,2]:
                        self.writer.add_scalar(f"win_charts_agent_{i}/{j}_position_all_prop",int(
                                                                                            self.hero_agents_list[i].position==j
                                                                                            ),(curren_epi+1))

                        self.writer.add_scalar(f"draw_charts_agent_{i}/{j}_place_in_draw",
                                                                   self.hero_agents_list[i].count_draw_dict[j],
                                                                   self.draw_count)

                        self.writer.add_scalar(f"draw_charts_agent_{i}/{j}_place_in_draw_ratio",
                                                                   self.hero_agents_list[i].count_draw_dict[j]/self.draw_count,
                                                                   self.draw_count)
                        
                    
            self.writer.add_scalar("draw_charts/draw_count",self.draw_count,self.global_step)
            self.writer.add_scalar("draw_charts/draw",1,(curren_epi+1))
            self.writer.add_scalar("draw_charts/draw_to_total_count",self.draw_count/(curren_epi +1+0.000001),self.global_step)
            
        else:
            
            non_draw_count =(curren_epi-self.draw_count+1+0.000001)
            for i in self.hero_agents_list:

                self.writer.add_scalar(f"win_charts_agent_{i}/position_win",self.hero_agents_list[i].position
                                                                                            ,(curren_epi+1))



                
                self.hero_agents_list[i].count_dict[self.hero_agents_list[i].position
                                               ] +=1

                
                for j in self.hero_agents_list[i].count_dict:
                    self.writer.add_scalar(f"win_charts_agent_{i}/{j}_position_prop",int(
                                                            self.hero_agents_list[i].position==j
                                                            ),(curren_epi+1))

                    self.writer.add_scalar(f"win_charts_agent_{i}/{j}_position",self.hero_agents_list[i].count_dict[j],
                                           (curren_epi+1))

                    self.writer.add_scalar(f"win_charts_agent_{i}/{j}_position_to_total_terminated",self.hero_agents_list[i].count_dict[j]/non_draw_count,(curren_epi+1))

                    if j not in [1,2]:
                        self.writer.add_scalar(f"win_charts_agent_{i}/{j}_position_all_prop",int(
                                                                                            self.hero_agents_list[i].position==j
                                                                                            ),(curren_epi+1))


            self.writer.add_scalar("draw_charts/draw",0,(curren_epi+1))
            
        for i in self.hero_agents_list:

            self.writer.add_scalar(f"win_charts_agent_{i}/position_all",self.hero_agents_list[i].position
                                                                                            ,(curren_epi+1))

            
            for j in self.hero_agents_list[i].count_dict:
                self.writer.add_scalar(f"win_charts_agent_{i}/{j}_position_to_total",(
                                                            self.hero_agents_list[i].count_dict[j]+
                                                            self.hero_agents_list[i].count_draw_dict[j]
                                                        
                                                            )/(curren_epi +1+0.00001 ),(curren_epi+1))#global_step)

            
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_count_per_episode",
                                               self.hero_agents_list[i].bad_move_count,self.global_step)
                
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_count_position_per_episode",
                                               self.hero_agents_list[i].bad_move_phase_count[0],self.global_step)
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_count_attack_per_episode",
                                               self.hero_agents_list[i].bad_move_phase_count[1],self.global_step)
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_count_fortify_per_episode",
                                               self.hero_agents_list[i].bad_move_phase_count[2],self.global_step)
            
            self.writer.add_scalar(f"moves/agent_{i}/total_moves",sum(
                                                    self.hero_agents_list[i].move_count.values()),self.global_step)
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_to_step_count_per_episode",
                                               self.hero_agents_list[i].bad_move_count/(sum(
                                                   self.hero_agents_list[i].move_count.values())+1),self.global_step)
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_to_step_position_per_episode",
                                               self.hero_agents_list[i].bad_move_phase_count[0]/( 
                                                   self.hero_agents_list[i].move_count[0]+1),self.global_step)
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_to_step_attack_per_episode",
                                               self.hero_agents_list[i].bad_move_phase_count[1]/( 
                                                   self.hero_agents_list[i].move_count[1]+1),self.global_step)
            self.writer.add_scalar(f"moves/agent_{i}/bad_move_to_step_fortify_per_episode",
                                               self.hero_agents_list[i].bad_move_phase_count[2]/( 
                                                   self.hero_agents_list[i].move_count[2]+1),self.global_step)

        
        self.writer.add_scalar("charts/epsilon",(curren_epi/((self.num_iterations*self.num_episodes)/10)),self.global_step)
        self.writer.add_scalar("charts/avg_per_epi_total_reward", np.mean(list(total_rewards.values())), self.global_step)

        

        #values_total = {i:0 for i in self.env.possible_agents}
        

        
        self.writer.add_scalar("charts/episodic_length", cur_epi_list[:step].sum(), self.global_step)
        
        for i in env.possible_agents:
            #cur_index = torch.where((current_agent[:,0] == i) &( cur_epi_list ))[0]

            #values_total[i] = values[cur_index].mean()
            #writer.add_scalar("charts/mean_value_per_epi_agent_"+str(i), values_total[i], global_step)
            
            self.writer.add_scalar("charts/total_reward_per_epi_agent_"+str(i), total_rewards[i], self.global_step)

    
    


        
        




In [90]:
T.args.num_minibatches

4

In [109]:

exp_3 = dict(
exp_name = 'exp3_ddqn_lr_bs_1',
learning_rate = 0.001,
batch_size = 512,
gamma = 0.99,
num_steps=120000,
num_iterations = 500,
episode_time_lim = 10000,
hero_agent_count = 2,
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 4
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.01
                 
                     #| {"render_mode" : None,"bad_mov_penalization" : 0.01,"render_":False#False
                     #                                   }
                 
                 )

)

exp_4 = dict(
exp_name = 'exp4_ddqn_2_agents_1_hero',
learning_rate = 0.001,
batch_size = 512,
gamma = 0.99,
num_steps=120000,
num_iterations = 500,
episode_time_lim = 10000,
hero_agent_count = 1,
env_config = dict(render_mode = None,#'rgb_array', 
                    default_attack_all  = True,
                    agent_count  = 2
                    ,use_placement_perc=True,
                    render_=False,
                    bad_mov_penalization = 0.01
                 )

)


In [110]:
#T = Trainer(Args,param_dict = exp_3)
T = Trainer(Args,param_dict = exp_4)

In [ ]:
T.run_training_loop()

done: {1: True, 2: True} ,total_reward: {1: 105.88000000000004, 2: -104.61999999999999} ,iteration: 36 ,episode: 0
done: {1: True, 2: True} ,total_reward: {1: 106.65000000000005, 2: -104.61999999999999} ,iteration: 36 ,episode: 1


In [89]:
env_config = dict(render_mode = 'rgb_array', default_attack_all  = True,
                render_ = True,agent_count  = 3,use_placement_perc=True)
env = env_risk(**(env_config | {"render_mode" : None,"verbose":False, "agent_count" :4,"render_":False}))


total_rewards = {i:0 for i in env.possible_agents}

step =0
for i__ in range(1):
    env.reset()#seed=42)
    #print(env_1.infos.keys())
    observation, reward, termination, truncation, info = env.last()
    
    total_rewards = {i:0 for i in env.possible_agents}
    
    #so there is an issue ...reward =2 sometimes ... doubling?

    for agent in env.agent_iter():
        step+=1
        e_t = env.terminations
        if sum(e_t.values()) <3:
            observation, reward, termination, truncation, info = env.last()
            
            action = env.action_space(agent).sample()
            
            #if env_1.phase_selection ==1:
            #    action = [int(action[0]),action[1]]
            part_0 =np.random.choice(np.where(env.board.calculated_action_mask(agent))[0])
            #print(part_0)
            
            #action = [part_0,np.around(action[1],2) #min(action[1],env_1.board.agents[agent].bucket ) 
            #                  if env_1.phase_selection==0 else  action[1]]
    
            action = [part_0,np.around(action[1],2)]
    
    
            for i in env.possible_agents:
                total_rewards[i]+=env.curr_rewards[i]
            
            env.step(action)

            if step%10000 ==0:
                print(step)
                print(env.board.territories)
                
                print('\naction',action,
                      '\ninfo',info,
                      '\ninfos',env.infos,
                      '\naction_valid',observation['action_mask'][action[0]],
                      '\nagent',agent,
                      '\nselected_agent',env.agent_selection,
                      '\ncurr_agent',env.board.current_agent,
                      '\ncurr phase',env.phase_selection,
                      '\nbad_trail count',env.board.bad_trials,
                      '\nmax_bad trails', env.board.max_bad_trials,
                      '\nreward',reward,
                      '\nreward',env.curr_rewards,
                      '\ntotal_reward',total_rewards,
                      '\nrewards',env.rewards,
                      '\nbuckets', [env.board.agents[i].bucket for i in env.agents]
                     )
                print('kill_list',env.kill_list,'term',e_t, 'buckets',[i.bucket for i in env.board.agents])
                #if sum(env.curr_rewards.values()) <0:
                #    input()
        else:
            print(env.board.territories)
            print('done',env.terminations.values(),env.curr_rewards)
            break
step


#need to debug .... so the action is valid... but there is no change
#so either the phase is not changing and the agent is not changing... why??


[[2 1.0]
 [2 1.0]
 [2 4.0]
 [2 4.0]
 [2 9.0]
 [2 1.0]
 [2 1.0]
 [2 4.0]
 [2 1.0]
 [2 1.0]]
done dict_values([True, True, True, True]) {1: -100, 2: 100, 3: -100, 4: -100}


3578